In [1]:
import pandas as pd
from openpyxl import load_workbook
import os

In [2]:
def read_sheet_regio(wb):
    
    sheet_regio_name = 'Regionalização'
    if sheet_regio_name not in wb.sheetnames:
        raise ValueError(f'File {file_path} sem aba regionalização')
    sheet_regio = wb[sheet_regio_name]
    regio_data = sheet_regio['B3':'C39']
    
    data = {'subprefeitura/zona' : [],
            'valor' : []}
    for row in regio_data:
        data['subprefeitura/zona'].append(row[0].value)
        data['valor'].append(row[1].value)
        
    return pd.DataFrame(data)

In [3]:
def read_dados_meta(wb):
    
    sheet_ficha_tecnica = 'Ficha técnica Meta '
    if sheet_ficha_tecnica not in wb.sheetnames:
        raise ValueError(f'File {file_path} sem aba ficha técnica')
    
    sheet_ficha = wb[sheet_ficha_tecnica]
    
    dados = {'secretaria' : sheet_ficha['C4'].value,
            'numero_meta' : sheet_ficha['C5'].value,
            'meta' : sheet_ficha['C6'].value,
            'indicador' : sheet_ficha['C9'].value}
    
    return dados

In [4]:
def get_dados_file(file_path):
    try:
        wb = load_workbook(file_path)
        dados_ficha = read_dados_meta(wb)
        dados_regio = read_sheet_regio(wb)

        for col, val in dados_ficha.items():
            dados_regio[col] = val

        return dados_regio
    except Exception as e:
        print(file_path)
        print(e)
        return pd.DataFrame()
        
    

In [5]:
def get_dados_all_files(data_dir):
    
    files = [os.path.join(data_dir, file) for file
            in os.listdir(data_dir) 
             if 'xls' in file.split('.')[-1]]
    
    dfs = []
    for file in files:
        dfs.append(get_dados_file(file))
    
    return pd.concat(dfs)

In [6]:
df = get_dados_all_files('original_data/fichas_metas/')

In [11]:
df.isnull().any()

subprefeitura/zona    False
valor                  True
secretaria            False
numero_meta           False
meta                  False
indicador             False
dtype: bool

In [13]:
df.to_csv('dados_regionalizacao_pdm.csv', encoding='utf-8', sep=';', quotechar='"', index=False)